In [1]:
import pandas as pd
import pickle
from sec_api import ExtractorApi
import dateutil.parser

extractorApi = ExtractorApi("YOUR API KEY")

In [2]:
CompanyTickers = [
    'AAPl',
    'TSLA', 
    'LULU',
    'SQ',
    'MSFT',
    'GOOG', 
    'KO',
    'XOM',
    'MCD',
    'GS',
    'JPM'
]

In [3]:
def getReports(filingsList):
    for url in filingsList:
        del url[0]
        filing_url = url[0]
        filing_date = url[1]
        dayofyear = dateutil.parser.parse(filing_date).strftime("%j")
        if url[-2] == '10-K':
            # get the original HTML of section 7 "Management’s Discussion and Analysis of Financial Condition and Results of Operations"

            section_html10K = extractorApi.get_section(filing_url, "7", "text")
            # insertVaribleIntoTable(dayofyear, company_name, section_html)
            url.append(section_html10K)
            url.append(dayofyear)
        elif url[-2] == '10-Q':
            # extract section 1A "Risk Factors" in part 2 as cleaned text
            section_html10Q = extractorApi.get_section(filing_url, "part1item2", "text")
            url.append(section_html10Q)
            url.append(dayofyear)
        else:
            pass
    filingsList = [x for x in filingsList if len(x) > 4]
    df = pd.DataFrame(filingsList, columns=['url', 'date', 'formType', 'ticker', 'MDA', 'dayofyear'])
    ticker = df.iloc[0]['ticker']
    df.to_csv(ticker + 'SEDAR.csv')
    

In [4]:
for ticker in CompanyTickers:
    fileString = ticker + '.csv'
    fileList = pd.read_csv(fileString).reset_index(drop=True).values.tolist()
    getReports(fileList)
    print(ticker, 'done')

JPM done
